In [1]:
from typing import List
from collections import defaultdict

import pandas as pd

# Impacto novo regulamento da UFRN

A UFRN aprovou o [novo regulamento da graduação](https://www.ufrn.br/imprensa/noticias/72045/consepe-aprova-novo-regulamento-da-graduacao), modificando a estrutura de notas. Esse notebook investiga qual o impacto do novo regulamento utilizando os dados abertos da UFRN para verificar se mais pessoas serão reprovadas.

A notícia da UFRN diz "Também foram realizadas várias simulações envolvendo os novos critérios de rendimento e, considerando as notas atuais dos estudantes, não foram verificados impactos significativos em relação à reprovação". Vamos investigar se o número de alunos aprovados se mantém com o novo regulamento, para isso vamos utilizar como amostra de dados o último semestre disponível na base de dados abertas: 2022.2

In [2]:
! wget https://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/b159805b-e7cb-4d71-872b-14a1a2625d7e/download/matriculas-2022.2.csv --no-check-certificate

--2023-07-04 20:53:43--  https://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/b159805b-e7cb-4d71-872b-14a1a2625d7e/download/matriculas-2022.2.csv
Resolving dados.ufrn.br (dados.ufrn.br)... 177.20.146.38
Connecting to dados.ufrn.br (dados.ufrn.br)|177.20.146.38|:443... connected.
  Issued certificate has expired.
HTTP request sent, awaiting response... 200 OK
Length: 67169203 (64M) [text/csv]
Saving to: ‘matriculas-2022.2.csv.2’

matriculas-2022.2.c 100%[===================>]  64,06M  2,93MB/s    in 19s     

2023-07-04 20:54:02 (3,37 MB/s) - ‘matriculas-2022.2.csv.2’ saved [67169203/67169203]



In [3]:
df_matriculas = pd.read_csv('matriculas-2022.2.csv', sep=';')
df_matriculas.head()

,id_turma,discente,id_curso,unidade,nota,reposicao,faltas_unidade,media_final,numero_total_faltas,descricao
0,57705399,1d568d48ce1a8b2ce4137e51abd0249a,2000005.0,1.0,"8,5",False,0.0,"8,7",0.0,APROVADO
1,57705399,1d568d48ce1a8b2ce4137e51abd0249a,2000005.0,2.0,"8,5",False,0.0,"8,7",0.0,APROVADO
2,57705399,1d568d48ce1a8b2ce4137e51abd0249a,2000005.0,3.0,"9,0",False,0.0,"8,7",0.0,APROVADO
3,57705399,5c30de64828a5c3e51e2a2428cb83165,2000005.0,1.0,"9,7",False,0.0,"9,6",0.0,APROVADO
4,57705399,5c30de64828a5c3e51e2a2428cb83165,2000005.0,2.0,"9,7",False,0.0,"9,6",0.0,APROVADO


Existem várias situações em que o discente pode se encontrar após a consolidação de uma disciplina. Abaixo são listados todas essas situações.

In [4]:
df_matriculas['descricao'].unique()

array(['APROVADO', 'DESISTENCIA', 'REPROVADO POR MÉDIA E POR FALTAS',
       'APROVADO POR NOTA', 'INDEFERIDO', 'TRANCADO', 'REPROVADO',
       'CANCELADO', 'EXCLUIDA', 'REPROVADO POR FALTAS',
       'REPROVADO POR NOTA E FALTA', 'REPROVADO POR NOTA', 'MATRICULADO',
       'DISPENSADO', 'EM ESPERA', 'AGUARDANDO DEFERIMENTO', 'CUMPRIU'],
      dtype=object)

Vamos investigar o impacto no novo regulamento sobre quem foi `Aprovado por Nota` (que pelo regulamento antigo teve média 5.0 com notas acima de 3.0 em cada unidade)

In [5]:
df_aprovados_nota = df_matriculas[df_matriculas['descricao'] == 'APROVADO POR NOTA'] 
len(df_aprovados_nota)

96129

É possível notar que 96129 matrículas constam como `Aprovado por Nota`. Vale ressaltar que esse número não é a quantidade de alunos que realmente foram aprovados por nota, uma vez que esses dados replicam as notas das unidades pagas pelo discente, então se uma disciplina tem 3 unidades o status de `Aprovado por Nota` aparece três vezes.

Para saber o real número de discentes vamos agrupar os dados por discente, turma e as unidades dessa disciplina.

In [6]:
df_aprovados_nota_grupo = df_aprovados_nota.groupby(by=['discente', 'id_turma', 'unidade'])
df_aprovados_nota_grupo.first()

id_curso nota reposicao  \
discente                         id_turma unidade                             
000098dc4f9897eccba03c5b1e9fb49a 57708907 1.0      2000122.0  6,7     False   
                                          2.0      2000122.0  5,6     False   
                                          3.0      2000122.0  5,2     False   
0000ad2567692a566a6d6f701eb9b161 57707687 1.0      2000014.0  6,0     False   
                                          2.0      2000014.0  7,0     False   
...                                                      ...  ...       ...   
fffc43b492fcefa5b99f7b6469834861 57707185 2.0      2000039.0  6,6     False   
                                          3.0      2000039.0  5,3     False   
                                 57707214 1.0      2000039.0  5,2     False   
                                          2.0      2000039.0  6,7     False   
                                          3.0      2000039.0  5,7     False   

                                                   faltas_unidade media_final  \
discente                         id_turma unidade                               
000098dc4f9897eccba03c5b1e9fb49a 57708907 1.0                 0.0         5,8   
                                          2.0                 0.0         5,8   
                                          3.0                 0.0         5,8   
0000ad2567692a566a6d6f701eb9b161 57707687 1.0                 0.0         6,5   
                                          2.0                 0.0         6,5   
...                                                           ...         ...   
fffc43b492fcefa5b99f7b6469834861 57707185 2.0                 0.0         6,1   
                                          3.0                 0.0         6,1   
                                 57707214 1.0                 0.0         5,9   
                                          2.0                 0.0         5,9   
                                          3.0                 0.0         5,9   

                                                   numero_total_faltas  \
discente                         id_turma unidade                        
000098dc4f9897eccba03c5b1e9fb49a 57708907 1.0                     14.0   
                                          2.0                     14.0   
                                          3.0                     14.0   
0000ad2567692a566a6d6f701eb9b161 57707687 1.0                     14.0   
                                          2.0                     14.0   
...                                                                ...   
fffc43b492fcefa5b99f7b6469834861 57707185 2.0                      4.0   
                                          3.0                      4.0   
                                 57707214 1.0                      2.0   
                                          2.0                      2.0   
                                          3.0                      2.0   

                                                           descricao  
discente                         id_turma unidade                     
000098dc4f9897eccba03c5b1e9fb49a 57708907 1.0      APROVADO POR NOTA  
                                          2.0      APROVADO POR NOTA  
                                          3.0      APROVADO POR NOTA  
0000ad2567692a566a6d6f701eb9b161 57707687 1.0      APROVADO POR NOTA  
                                          2.0      APROVADO POR NOTA  
...                                                              ...  
fffc43b492fcefa5b99f7b6469834861 57707185 2.0      APROVADO POR NOTA  
                                          3.0      APROVADO POR NOTA  
                                 57707214 1.0      APROVADO POR NOTA  
                                          2.0      APROVADO POR NOTA  
                                          3.0      APROVADO POR NOTA  

[65071 rows x 7 columns]

É possível notar que o impacto desse novo regulamento se daria sobre `65071` estudantes de `2022.2`. Mas qual será o impacto? Para medir isso vamos reprocessar as notas dos alunos com o novo regulamento e ver quantos estudantes continuam se enquadrando na situação de `Aprovados por nota`.

In [7]:
def unidades_acima_quatro(notas: List[float]) -> bool:
    return all([nota >= 4.0 for nota in notas])

def media_acima_seis(notas: List[float]) -> bool:
    media = sum([nota for nota in notas])/len(notas)
    return media >= 6.0

Para facilitar o processamento vamos modificar a estrutura de dados para um dicionário python no formato: `{(discente, turma): [nota unidade 1, nota unidade 2, ..., nota unidade x]`

In [8]:
preprocessamento_afetados = defaultdict(lambda: [])

In [9]:
for (discente, turma, unidade), group in df_aprovados_nota_grupo:
    nota = float(group['nota'].iloc[0].replace(',', '.'))
    preprocessamento_afetados[(discente, turma)].append(nota)

In [10]:
aprovados_novo_regulamento = 0
for (discente, turma), notas in preprocessamento_afetados.items():
    if unidades_acima_quatro(notas) and media_acima_seis(notas):
        aprovados_novo_regulamento += 1

Com o calculo anterior podemos concluir que o seguinte número de alunos deixariam de passar por nota com o novo regulamento.

In [11]:
len(df_aprovados_nota_grupo) - aprovados_novo_regulamento

55703